## TASK 1

In [1]:
import pandas as pd
import numpy as np

In [2]:
#merge the dataframes

url_a_test = 'Data/arguments-test.tsv'
df_a_test = pd.read_csv(url_a_test, sep='\t')

url_a_training = 'Data/arguments-training.tsv'
df_a_training = pd.read_csv(url_a_training, sep='\t')

url_a_validation = 'Data/arguments-validation.tsv'
df_a_validation = pd.read_csv(url_a_validation, sep='\t')

url_l_test = 'Data/labels-test.tsv'
df_l_test = pd.read_csv(url_l_test, sep='\t')

url_l_training = 'Data/labels-training.tsv'
df_l_training = pd.read_csv(url_l_training, sep='\t')

url_l_validation = 'Data/labels-validation.tsv'
df_l_validation = pd.read_csv(url_l_validation, sep='\t')


In [3]:
#merge argument dataframes with label dataframes
df_test = pd.merge(df_a_test, df_l_test, on='Argument ID')

df_training = pd.merge(df_a_training, df_l_training, on='Argument ID')

df_validation = pd.merge(df_a_validation, df_l_validation, on='Argument ID')

In [4]:
#merge level 2 annotations to level 3 categories

df_test['Openess to change'] = df_test['Self-direction: thought'] | df_test['Self-direction: action'] | df_test['Stimulation'] | df_test['Hedonism'] 
df_test['Self-enhancement'] = df_test['Hedonism'] | df_test['Achievement'] | df_test['Power: dominance'] | df_test['Power: resources'] | df_test['Face']
df_test['Conservation'] = df_test['Face'] | df_test['Security: personal'] | df_test['Security: societal'] | df_test['Tradition'] | df_test['Conformity: rules'] | df_test['Conformity: interpersonal'] | df_test['Humility']
df_test['Self-transcendence'] = df_test['Humility'] | df_test['Benevolence: caring'] | df_test['Benevolence: dependability'] | df_test['Universalism: concern'] | df_test['Universalism: nature']| df_test['Universalism: tolerance'] | df_test['Universalism: objectivity']

df_training['Openess to change'] = df_training['Self-direction: thought'] | df_training['Self-direction: action'] | df_training['Stimulation'] | df_training['Hedonism']
df_training['Self-enhancement'] = df_training['Hedonism'] | df_training['Achievement'] | df_training['Power: dominance'] | df_training['Power: resources'] | df_training['Face']
df_training['Conservation'] = df_training['Face'] | df_training['Security: personal'] | df_training['Security: societal'] | df_training['Tradition'] | df_training['Conformity: rules'] | df_training['Conformity: interpersonal'] | df_training['Humility']
df_training['Self-transcendence'] = df_training['Humility'] | df_training['Benevolence: caring'] | df_training['Benevolence: dependability'] | df_training['Universalism: concern'] | df_training['Universalism: nature']| df_training['Universalism: tolerance'] | df_training['Universalism: objectivity']

df_validation['Openess to change'] = df_validation['Self-direction: thought'] | df_validation['Self-direction: action'] | df_validation['Stimulation'] | df_validation['Hedonism']
df_validation['Self-enhancement'] = df_validation['Hedonism'] | df_validation['Achievement'] | df_validation['Power: dominance'] | df_validation['Power: resources'] | df_validation['Face']
df_validation['Conservation'] = df_validation['Face'] | df_validation['Security: personal'] | df_validation['Security: societal'] | df_validation['Tradition'] | df_validation['Conformity: rules'] | df_validation['Conformity: interpersonal'] | df_validation['Humility']
df_validation['Self-transcendence'] = df_validation['Humility'] | df_validation['Benevolence: caring'] | df_validation['Benevolence: dependability'] | df_validation['Universalism: concern'] | df_validation['Universalism: nature']| df_validation['Universalism: tolerance'] | df_validation['Universalism: objectivity']

df_validation.head()

,Argument ID,Conclusion,Stance,Premise,Self-direction: thought,Self-direction: action,Stimulation,Hedonism,Achievement,Power: dominance,...,Benevolence: caring,Benevolence: dependability,Universalism: concern,Universalism: nature,Universalism: tolerance,Universalism: objectivity,Openess to change,Self-enhancement,Conservation,Self-transcendence
0,A01001,Entrapment should be legalized,in favor of,if entrapment can serve to more easily capture...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,A01012,The use of public defenders should be mandatory,in favor of,the use of public defenders should be mandator...,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,A02001,Payday loans should be banned,in favor of,payday loans create a more impoverished societ...,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,1
3,A02002,Surrogacy should be banned,against,Surrogacy should not be banned as it is the wo...,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,A02009,Entrapment should be legalized,against,entrapment is gravely immoral and against huma...,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,1,1


In [5]:
#drop unuseful columns
df_test = df_test.drop(columns=['Self-direction: thought', 'Self-direction: action', 'Stimulation', 'Hedonism', 'Achievement', 'Power: dominance', 'Power: resources', 'Face', 'Security: personal', 'Security: societal', 'Tradition', 'Conformity: rules', 'Conformity: interpersonal', 'Humility', 'Benevolence: caring', 'Benevolence: dependability', 'Universalism: concern', 'Universalism: nature', 'Universalism: tolerance', 'Universalism: objectivity'])
df_training = df_training.drop(columns=['Self-direction: thought', 'Self-direction: action', 'Stimulation', 'Hedonism', 'Achievement', 'Power: dominance', 'Power: resources', 'Face', 'Security: personal', 'Security: societal', 'Tradition', 'Conformity: rules', 'Conformity: interpersonal', 'Humility', 'Benevolence: caring', 'Benevolence: dependability', 'Universalism: concern', 'Universalism: nature', 'Universalism: tolerance', 'Universalism: objectivity'])
df_validation = df_validation.drop(columns=['Self-direction: thought', 'Self-direction: action', 'Stimulation', 'Hedonism', 'Achievement', 'Power: dominance', 'Power: resources', 'Face', 'Security: personal', 'Security: societal', 'Tradition', 'Conformity: rules', 'Conformity: interpersonal', 'Humility', 'Benevolence: caring', 'Benevolence: dependability', 'Universalism: concern', 'Universalism: nature', 'Universalism: tolerance', 'Universalism: objectivity'])

In [9]:
df_validation.head()

,Argument ID,Conclusion,Stance,Premise,Openess to change,Self-enhancement,Conservation,Self-transcendence
0,A01001,Entrapment should be legalized,in favor of,if entrapment can serve to more easily capture...,0,0,1,0
1,A01012,The use of public defenders should be mandatory,in favor of,the use of public defenders should be mandator...,0,0,0,1
2,A02001,Payday loans should be banned,in favor of,payday loans create a more impoverished societ...,0,0,1,1
3,A02002,Surrogacy should be banned,against,Surrogacy should not be banned as it is the wo...,1,0,0,0
4,A02009,Entrapment should be legalized,against,entrapment is gravely immoral and against huma...,0,0,1,1


## TASK 2 - MODEL DEFINITION

In [6]:
#array of categories 
categories = ['Openess to change', 'Self-enhancement', 'Conservation', 'Self-transcendence']

#dictionary of classifiers
classifiers = {}

### Baseline models

In [10]:

def create_random_uniform_classifier(category):
    """
    Crea un classificatore casuale uniforme che predice 0 o 1.

    :param category: La categoria che il classificatore predirà
    :return: Una funzione che genera previsioni casuali uniformi
    """
    def random_uniform_classifier(size):
        """
        Genera previsioni casuali uniformi.

        :param size: Numero di previsioni da generare
        :return: Array di previsioni casuali uniformi
        """
        return np.random.choice([0, 1], size=size)
    
    return random_uniform_classifier


In [11]:
def create_majority_classifier(category, majority_value):
    """
    Crea un classificatore di maggioranza che predice sempre il valore di maggioranza.

    :param category: La categoria che il classificatore predirà
    :param majority_value: Il valore di maggioranza (0 o 1)
    :return: Una funzione che genera previsioni di maggioranza
    """
    def majority_classifier(size):
        """
        Genera previsioni di maggioranza.

        :param size: Numero di previsioni da generare
        :return: Array di previsioni di maggioranza
        """
        return np.full(size, majority_value)
    
    return majority_classifier

In [13]:
#create classifiers for each category and save them in the dictionary
for category in categories:
    #random uniform classifier
    random_uniform_name = f'random_uniform_classifier_{category}'
    classifiers[random_uniform_name] = create_random_uniform_classifier(category)

    #majority classifier
    majority_name = f'majority_classifier_{category}'
    classifiers[majority_name] = create_majority_classifier(category, majority_value=1) #da capire perchè majority_value=1

In [14]:
print(classifiers)

{'random_uniform_classifier_Openess to change': <function create_random_uniform_classifier.<locals>.random_uniform_classifier at 0x13bfbd940>, 'majority_classifier_Openess to change': <function create_majority_classifier.<locals>.majority_classifier at 0x13bfbdb20>, 'random_uniform_classifier_Self-enhancement': <function create_random_uniform_classifier.<locals>.random_uniform_classifier at 0x13bfbdc60>, 'majority_classifier_Self-enhancement': <function create_majority_classifier.<locals>.majority_classifier at 0x13bfbd9e0>, 'random_uniform_classifier_Conservation': <function create_random_uniform_classifier.<locals>.random_uniform_classifier at 0x13bfbdd00>, 'majority_classifier_Conservation': <function create_majority_classifier.<locals>.majority_classifier at 0x13bfbdda0>, 'random_uniform_classifier_Self-transcendence': <function create_random_uniform_classifier.<locals>.random_uniform_classifier at 0x13bfbde40>, 'majority_classifier_Self-transcendence': <function create_majority_cl

In [ ]:
#Applying the classifiers to the test set

#iterating over the classifiers
for classifier_name, classifier in classifiers.items():
    #defining the category from the classifier name
    category = classifier_name.split('_')[-1]
    
    #applying the classifier to the test set
    predictions = classifier(size=len(df_test))
    
    #saving the predictions in the test set
    df_test[f'predicted_{classifier_name}'] = predictions

#saving the test set with the predictions
print(df_test)

### BERT models

In [7]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch

2024-07-26 18:33:13.370106: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### Columns encoding
Mi sono accorta dopo che le colonne devono essere encoded, ho provato a fare una prima funzione ma non funziona 

In [15]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [19]:
#define the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('premise', CountVectorizer(), 'Premise'),
        ('conclusion', CountVectorizer(), 'Conclusion'),
        ('stance', OneHotEncoder(), 'Stance')
    ])

# Applicare il ColumnTransformer al DataFrame
df_transformed = preprocessor.fit_transform(df_test)


ValueError: 1D data passed to a transformer that expects 2D data. Try to specify the column selection as a list of one item instead of a scalar.

In [8]:
#BERT w/C model creation
def create_bert_classifier(category):
    """
    Crea un classificatore basato su BERT per una categoria specifica.

    :param category: La categoria per cui creare il classificatore
    :return: Una funzione che utilizza BERT per classificare il testo
    """
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

    def bert_classifier(texts):
        """
        Classifica i testi utilizzando il modello BERT.

        :param texts: Lista di testi da classificare
        :return: Array di previsioni
        """
        inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)
        return predictions.numpy()
    
    return bert_classifier


In [9]:
#creating BERT_w/C classifiers for each category and saving them in the dictionary
for category in categories:
    bert_name = f'BERT_w/C_{category}'
    classifiers[bert_name] = create_bert_classifier(category)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [10]:
#apply the classifiers to the test set
for classifier_name, classifier in classifiers.items():
    predictions = classifier(df_test['Conclusion'].tolist())
    df_test[f'predicted_{classifier_name}'] = predictions

print(df_test)

     Argument ID                                         Conclusion  \
0         A26004                   We should end affirmative action   
1         A26010                   We should end affirmative action   
2         A26016                          We should ban naturopathy   
3         A26024                 We should prohibit women in combat   
4         A26026                          We should ban naturopathy   
...          ...                                                ...   
1571      E07272             We should end mass migration together.   
1572      E07273             We should end mass migration together.   
1573      E07275  We should consider Russian interests in the EU...   
1574      E07280  We should adopt an extension of the applicatio...   
1575      E08023               We should abolish covid digital pass   

           Stance                                            Premise  \
0         against   affirmative action helps with employment equity.   
1  

In [11]:
#BERT w/CP model creation 
def create_bert_classifier_with_premise(category):
    """
    Crea un classificatore basato su BERT per una categoria specifica utilizzando sia Conclusion che Premise.

    :param category: La categoria per cui creare il classificatore
    :return: Una funzione che utilizza BERT per classificare il testo
    """
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

    def bert_classifier(conclusions, premises):
        """
        Classifica i testi utilizzando il modello BERT.

        :param conclusions: Lista di conclusioni da classificare
        :param premises: Lista di premesse da classificare
        :return: Array di previsioni
        """
        combined_texts = [f"{premise} [SEP] {conclusion}" for premise, conclusion in zip(premises, conclusions)]
        inputs = tokenizer(combined_texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)
        return predictions.numpy()
    
    return bert_classifier

In [12]:
#creating BERT_w/CP classifiers for each category and saving them in the dictionary
for category in categories:
    bert_name = f'BERT_w/CP_{category}'
    classifiers[bert_name] = create_bert_classifier_with_premise(category)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [ ]:
#BERT w/CPS model creation 
def create_bert_classifier_with_premise_and_stance(category):
    """
    Crea un classificatore basato su BERT per una categoria specifica utilizzando Conclusion, Premise e Stance.

    :param category: La categoria per cui creare il classificatore
    :return: Una funzione che utilizza BERT per classificare il testo
    """
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

    def bert_classifier(conclusions, premises, stances):
        """
        Classifica i testi utilizzando il modello BERT.

        :param conclusions: Lista di conclusioni da classificare
        :param premises: Lista di premesse da classificare
        :param stances: Lista di stance da classificare
        :return: Array di previsioni
        """
        combined_texts = [f"{premise} [SEP] {conclusion} [SEP] {stance}" for premise, conclusion, stance in zip(premises, conclusions, stances)]
        inputs = tokenizer(combined_texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)
        return predictions.numpy()
    
    return bert_classifier


In [ ]:
#create BERT_w/CPS classifiers for each category and save them in the dictionary
for category in categories:
    bert_name = f'BERT_w/CPS_{category}'
    classifiers[bert_name] = create_bert_classifier_with_premise_and_stance(category)


In [ ]:
#apply the BERT w_CPS classifiers to the test set
for classifier_name, classifier in classifiers.items():
    predictions = classifier(df_test['Conclusion'].tolist(), df_test['Premise'].tolist(), df_test['Stance'].tolist())
    df_test[f'predicted_{classifier_name}'] = predictions

Cercare di capire come fare il training di BERT e se per ora è corretto